In [2]:
import os
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [4]:
feature_extractor = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

/tmp/ipykernel_70848/944957297.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  feature_extractor = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


In [5]:
def extract_features(img_path):
    """একটি ইমেজ থেকে CNN feature বের করে"""
    try:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = feature_extractor.predict(x, verbose=0)
        return features.flatten()
    except:
        return None

In [6]:
train_dir = 'data/train'
X_train, y_train = [], []

for person_name in os.listdir(train_dir):
    person_folder = os.path.join(train_dir, person_name)
    if not os.path.isdir(person_folder):
        continue
    
    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        features = extract_features(img_path)
        if features is not None:
            X_train.append(features)
            y_train.append(person_name)

X_train = np.array(X_train)
y_train = np.array(y_train)

print("Training features shape:", X_train.shape)
print("Total persons:", len(set(y_train)))

Training features shape: (15, 1280)
Total persons: 4


In [7]:
knn_clf = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_clf.fit(X_train, y_train)

print("KNN model trained successfully!")

KNN model trained successfully!


In [8]:
test_dir = 'data/test'

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    features = extract_features(img_path)
    
    if features is not None:
        prediction = knn_clf.predict([features])[0]
        distance, index = knn_clf.kneighbors([features], n_neighbors=1, return_distance=True)
        print(f"Image: {img_name} → Predicted Person: {prediction} (Distance: {distance[0][0]:.2f})")
    else:
        print(f"Could not extract features for: {img_name}")

Image: test_person4.jpeg → Predicted Person: person1 (Distance: 17.89)
Image: p.jpg → Predicted Person: person1 (Distance: 17.74)
Image: test_person3.png → Predicted Person: person1 (Distance: 21.78)
Image: test_person2.jpeg → Predicted Person: person1 (Distance: 18.72)
Image: test_person1.jpeg → Predicted Person: person1 (Distance: 22.80)
Image: m.webp → Predicted Person: person2 (Distance: 18.69)
